In [20]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import scipy.stats as stats
import statsmodels.api as sm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt
from statsmodels.multivariate.pca import PCA as PCA

In [58]:
SECTORS=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='B:K'))
MARKET=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='L'))
RF=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='M'))
FUNDS=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='N:O'))

## Part 1 

In [63]:
n_sim=1000
ret_sim=np.zeros((len(SECTORS),n_sim))
stats_sim=np.zeros((3,n_sim))

for i in range(n_sim):
    #print(i)
    score_t=np.random.normal(0,1,SECTORS.shape)
    for t in range(len(SECTORS)):
        select=score_t[t,:]>=np.quantile(score_t[t,:],0.7) 
        ret_sim[t,i]=np.mean(SECTORS[t,select])

In [98]:
n_sim=1000 #1000 simulations
ret_sim=np.zeros((len(SECTORS),n_sim)) 
stats_sim=np.zeros((3,n_sim)) 

for i in range(n_sim):
    #print(i)
    score_t=np.random.normal(0,1,SECTORS.shape) #distribution loi normale
    t=0 #chaque ligne/date on long le stock si > quantile 
    select=score_t[t,:]>=np.quantile(score_t[t,:],0.7) #>=0.7 bc we want to invest in 3 stocks
    ret_sim[t,i]=np.mean(SECTORS[t,select])

In [99]:
np.quantile(score_t[t,:],0.7)

0.5990272269670478

In [100]:
score_t[t,:]

array([-0.08390569,  0.59255227, -1.26829546,  0.50528454, -0.07441422,
        1.52728333, -0.29323115,  0.47622937, -0.49001984,  0.62492706])

In [101]:
score_t = pd.DataFrame(score_t)
score_t

,0,1,2,3,4,5,6,7,8,9
0,-0.083906,0.592552,-1.268295,0.505285,-0.074414,1.527283,-0.293231,0.476229,-0.490020,0.624927
1,1.021002,-0.259839,-0.058374,0.242747,-1.471658,0.610375,0.110910,-1.452028,1.081030,-1.912821
2,-0.705085,-0.830635,-1.368855,-1.351352,-0.197509,1.492915,-0.141562,1.179305,1.466966,-0.619269
3,-1.580993,-1.750237,1.512768,-0.547469,-1.214986,-0.035010,1.627120,0.054138,-0.400760,-0.531734
4,-0.596886,0.130300,-0.613723,-0.832252,1.913458,-1.248393,0.715376,-1.423481,-0.901533,0.557248
...,...,...,...,...,...,...,...,...,...,...
190,1.557502,0.947669,0.619379,0.335494,2.063582,1.019162,3.311214,-1.624678,-0.338138,1.491745
191,-0.009405,-0.233956,-0.804672,1.009843,-0.205436,-0.320962,-2.060903,-0.360488,-0.778401,0.321466
192,-0.194827,1.113806,-0.257535,1.972124,-2.163395,0.697861,-2.116109,0.405944,-0.280971,-0.058427
193,-0.112169,-1.344523,1.151603,-1.184935,1.232082,0.867741,0.672508,0.987980,1.414020,0.971243


In [102]:
select = pd.DataFrame(select)
select

,0
0,False
1,False
2,False
3,False
4,False
5,True
6,False
7,False
8,False
9,True


## Part 2

In [23]:
# annualized returns        
stats_sim[0,:]=np.mean(ret_sim,axis=0)*12

In [25]:
# Sharpe ratios
stats_sim[1,:]=(np.mean(ret_sim,axis=0)*12-np.mean(RF))/(np.std(ret_sim,axis=0)*12**0.5)

In [26]:
# alpha vs. SP500
for i in range(n_sim):
    model = sm.OLS(ret_sim[:,i],sm.add_constant(MARKET))
    results = model.fit()
    stats_sim[2,i]=results.params[0]*12

In [27]:
### STATISTICS LUCKY THRESHOLDS
threshlods=np.quantile(stats_sim,0.99,axis=1)

In [29]:
threshlods

array([0.13935   , 0.81867773, 0.03666005])

In [28]:
### FUND STATS
print("Annualized performances", np.mean(FUNDS,axis=0)*12)
print("Sharpe Ratios", (np.mean(FUNDS,axis=0)*12-np.mean(RF))/(np.std(FUNDS,axis=0)*12**0.5))
model = sm.OLS(FUNDS[:,0],sm.add_constant(MARKET))
results = model.fit()
print("Alpha fund 1:", results.params[0]*12)
model = sm.OLS(FUNDS[:,1],sm.add_constant(MARKET))
results = model.fit()
print("Alpha fund 1:", results.params[0]*12)

Annualized performances [0.13613736 0.16684007]
Sharpe Ratios [0.80895556 1.02190067]
Alpha fund 1: 0.028952095544430796
Alpha fund 1: 0.0654962772230828


In [41]:
SECTORS=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='B:K'))
MARKET=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='L'))
RF=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='M'))
FUND_1=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='N'))
FUND_2=np.array(pd.read_excel('/Users/Alicia/Documents/:M2 203/S2/Advanced Asset Management/Monarcha/HOMEWORK_3/HOMEWORK_3.xlsx', 'Feuil1',usecols='O'))

In [42]:
### FUND 1 STATS
print("Annualized performances", np.mean(FUND_1,axis=0)*12)
print("Sharpe Ratios", (np.mean(FUND_1,axis=0)*12-np.mean(RF))/(np.std(FUND_1,axis=0)*12**0.5))
model = sm.OLS(FUND_1,sm.add_constant(MARKET))
results = model.fit()
print("Alpha fund 1:", results.params[0]*12)

Annualized performances [0.13613736]
Sharpe Ratios [0.80895556]
Alpha fund 1: 0.028952095544430796


In [43]:
### FUND 2 STATS
print("Annualized performances", np.mean(FUND_2,axis=0)*12)
print("Sharpe Ratios", (np.mean(FUND_2,axis=0)*12-np.mean(RF))/(np.std(FUND_2,axis=0)*12**0.5))
model = sm.OLS(FUND_2,sm.add_constant(MARKET))
results = model.fit()
print("Alpha fund 1:", results.params[0]*12)

Annualized performances [0.16684007]
Sharpe Ratios [1.02190067]
Alpha fund 1: 0.0654962772230828


## Part 3

In [ ]:
#luck for fund 1 (inside /< treshold) but not for fund 2